# Ray Crash Course - Exercise Solutions

This notebook discusses solutions for the exercises in the _crash course_.

## 01 Ray Crash Course - Tasks - Exercise 1

As currently written, the memory footprint of `estimate_pi` scales linearly with `N`, because it allocates two NumPy arrays of size `N`. This limits the size of `N` we can evaluate (as I confirmed by locking up my laptop...). However, this isn't actually necessary. We could do the same calculation in "blocks, for example `m` blocks of size `N/m` and then combine the results. Furthermore, there's no dependencies between the calculations with those blocks, giving us further potential speed-up by parellelizing them with Ray.

Adapt `ray_estimate_pi` to use this technique. Pick some `N` value above which the calculation is done in blocks. Compare the performance of the old vs. new implementation. 

As you do this exercise, you might ponder the fact that we often averaged multiple trials for a given `N` and then ask yourself, what's the difference between averaging `10` trials for `N = 1000` vs. `1` trial for `N = 10000`, for example?

First, import things we need and redefine functions and data we need from the notebook:

In [39]:
import numpy as np
import sys, time, statistics, math
import ray
sys.path.append('..')
from pi_calc import str_large_n

In [2]:
trials = 5

In [3]:
ray.init(address='auto')

2020-04-30 12:09:01,221	INFO resource_spec.py:212 -- Starting Ray with 4.54 GiB memory available for workers and up to 2.29 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-04-30 12:09:01,547	INFO services.py:1148 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '192.168.1.149',
 'redis_address': '192.168.1.149:51488',
 'object_store_address': '/tmp/ray/session_2020-04-30_12-09-01_213619_3147/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-04-30_12-09-01_213619_3147/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-04-30_12-09-01_213619_3147'}

In [4]:
print(f'Dashboard URL: http://{ray.get_webui_url()}')

Dashboard URL: http://localhost:8265


Here's `estimate_pi` again, but now we'll also return the counts, for reasons we'll discuss shortly.

In [56]:
def estimate_pi(num_samples):
    xs = np.random.uniform(low=-1.0, high=1.0, size=num_samples)   # Generate num_samples random samples for the x coordinate.
    ys = np.random.uniform(low=-1.0, high=1.0, size=num_samples)   # Generate num_samples random samples for the y coordinate.
    xys = np.stack((xs, ys), axis=-1)                              # Like Python's "zip(a,b)"; creates np.array([(x1,y1), (x2,y2), ...]).
    inside = xs*xs + ys*ys <= 1.0                                  # Creates a predicate over all the array elements.
    xys_inside = xys[inside]                                       # Selects only those "zipped" array elements inside the circle.
    in_circle = xys_inside.shape[0]                                # Return the number of elements inside the circle.
    approx_pi = 4.0*in_circle/num_samples                          # The Pi estimate.
    return approx_pi, in_circle, num_samples

Here's the original `ray_estimate_pi`, but now it will also return the counts, not just $\pi$. 

In [57]:
@ray.remote
def ray_estimate_pi(num_samples):
    return estimate_pi(num_samples)

In [58]:
fmt = '{:10.5f} seconds: pi ~ {:7.6f}, stddev = {:5.4f}, error = {:5.4f}%'

Here's `ray_try_it`, but now we handle the additional returned values from `ray_estimate_pi`:

In [109]:
def ray_try_it(n, trials):
    print('trials = {:5d}, N = {:s}: '.format(trials, str_large_n(n, padding=15)), end='')   # str_large_n imported above.
    start = time.time()
    ids = [ray_estimate_pi.remote(n) for _ in range(trials)]
    pis_counts = ray.get(ids)
    pis = list(map(lambda t: t[0], pis_counts))
    approx_pi = statistics.mean(pis)
    stdev = 0.0 if trials == 1 else statistics.stdev(pis)
    duration = time.time() - start
    error = (100.0*abs(approx_pi-np.pi)/np.pi)
    print(fmt.format(duration, approx_pi, stdev, error))   # str_large_n imported above.
    return trials, n, duration, approx_pi, stdev, error

First, let's look at the "ponder" question at the end, just using the original implementation. We'll do a few runs of the following cell:

In [93]:
for n in [1000, 10000, 100000, 1000000, 10000000]:
    ray_try_it(n, round(10000000/n))

trials = 10000, N =           1,000:    2.11654 seconds: pi ~ 3.142452, stddev = 0.0513, error = 0.0274%
trials =  1000, N =          10,000:    0.23774 seconds: pi ~ 3.142216, stddev = 0.0162, error = 0.0198%
trials =   100, N =         100,000:    0.12214 seconds: pi ~ 3.141495, stddev = 0.0050, error = 0.0031%
trials =    10, N =       1,000,000:    0.17723 seconds: pi ~ 3.141891, stddev = 0.0019, error = 0.0095%
trials =     1, N =      10,000,000:    0.64956 seconds: pi ~ 3.141460, stddev = 0.0000, error = 0.0042%


In [94]:
for n in [1000, 10000, 100000, 1000000, 10000000]:
    ray_try_it(n, round(10000000/n))

trials = 10000, N =           1,000:    2.21157 seconds: pi ~ 3.142165, stddev = 0.0520, error = 0.0182%
trials =  1000, N =          10,000:    0.25916 seconds: pi ~ 3.141835, stddev = 0.0167, error = 0.0077%
trials =   100, N =         100,000:    0.13249 seconds: pi ~ 3.142105, stddev = 0.0056, error = 0.0163%
trials =    10, N =       1,000,000:    0.17565 seconds: pi ~ 3.141208, stddev = 0.0020, error = 0.0122%
trials =     1, N =      10,000,000:    0.65809 seconds: pi ~ 3.141775, stddev = 0.0000, error = 0.0058%


In [95]:
for n in [1000, 10000, 100000, 1000000, 10000000]:
    ray_try_it(n, round(10000000/n))

trials = 10000, N =           1,000:    2.15662 seconds: pi ~ 3.142175, stddev = 0.0523, error = 0.0185%
trials =  1000, N =          10,000:    0.23714 seconds: pi ~ 3.141688, stddev = 0.0163, error = 0.0030%
trials =   100, N =         100,000:    0.13763 seconds: pi ~ 3.141859, stddev = 0.0055, error = 0.0085%
trials =    10, N =       1,000,000:    0.17017 seconds: pi ~ 3.141774, stddev = 0.0021, error = 0.0058%
trials =     1, N =      10,000,000:    0.65895 seconds: pi ~ 3.141655, stddev = 0.0000, error = 0.0020%


The standard deviation is misleading now, because the number of trials change. The errors are roughly within an order of magnitude, due in part to expected statistical variation. Generally speaking, larger `N` and lower `trials` had lower errors. This may be due to the other big source of variation, the inevitable rounding error computing $\pi$ (`4 * inside_count/N`), one time per trial (`1` to `10,000` times). Experiments are supposed to eliminate as many extraneous variables as possible, so I would argue that sticking to one value for `trials` and varying `N` is more meaningful. In fact, in the implementation that follows, we'll eliminate the potential rounding error variation by keep track of the inside and total counts, then computing $\pi$ once at the end.

First, a function to return sample sizes for a given `N` and `m`.

In [52]:
def sample_sizes(N, m):
    ranges = [(m*i, m*(i+1)) for i in range(math.ceil(N/m))]
    if ranges[-1][1] > N:
        ranges[-1] = (ranges[-1][0], N)
    return list(map(lambda x: x[1]-x[0], ranges))

In [70]:
@ray.remote
def ray_estimate_pi_blocks(num_samples, m):
    """
    Perform the estimate in blocks up to ``m`` samples in size. A more user-friendly solution would embed logic to
    determine an reasonably good ``m`` value, but for our purposes, passing in ``m`` is more convenient.
    """
    sizes = sample_sizes(num_samples, m)
    ids = [ray_estimate_pi.remote(size) for size in sizes]
    values = ray.get(ids)  # Not using ray.wait() is okay; the tasks are all roughly the same size
    inside_count = 0
    total_count = 0
    for _, icount, tcount in values:    # Toss the pi value returned
        inside_count += icount
        total_count += tcount
    return 4.0*inside_count/total_count, inside_count, total_count

Let's try it:

In [82]:
for m in [10000, 100000, 1000000]:
    print(f'm = {m}:')
    for n in [1000, 10000, 100000, 1000000, 10000000, 100000000]:
        start = time.time()
        approx_pi, inside_count, total_count = ray.get(ray_estimate_pi_blocks.remote(n, 100000))
        duration = time.time() - start
        print(f'{n:15}: duration = {duration:6.5} seconds, pi = {approx_pi:6.5}, # inside/outside = {inside_count:12}/{total_count}')

m = 10000:
           1000: duration = 0.0044899 seconds, pi =  3.108, # inside/outside =          777/1000
          10000: duration = 0.0027683 seconds, pi = 3.1544, # inside/outside =         7886/10000
         100000: duration = 0.007292 seconds, pi = 3.1398, # inside/outside =        78495/100000
        1000000: duration = 0.017121 seconds, pi = 3.1414, # inside/outside =       785359/1000000
       10000000: duration = 0.12724 seconds, pi = 3.1412, # inside/outside =      7853049/10000000
      100000000: duration = 1.1888 seconds, pi = 3.1416, # inside/outside =     78541126/100000000
m = 100000:
           1000: duration = 0.0024719 seconds, pi =  3.132, # inside/outside =          783/1000
          10000: duration = 0.0020089 seconds, pi = 3.1368, # inside/outside =         7842/10000
         100000: duration = 0.0055439 seconds, pi =  3.149, # inside/outside =        78725/100000
        1000000: duration = 0.013774 seconds, pi = 3.1419, # inside/outside =       785463/10

Let's compare to the original implementation:

In [83]:
for n in [1000, 10000, 100000, 1000000, 10000000, 100000000]:
    start = time.time()
    approx_pi, inside_count, total_count = ray.get(ray_estimate_pi.remote(n))
    duration = time.time() - start
    print(f'{n:15}: duration = {duration:6.5} seconds, pi = {approx_pi:6.5}, # inside/outside = {inside_count:12}/{total_count}')

           1000: duration = 0.0014429 seconds, pi =  3.152, # inside/outside =          788/1000
          10000: duration = 0.022624 seconds, pi = 3.1488, # inside/outside =         7872/10000
         100000: duration = 0.03082 seconds, pi = 3.1402, # inside/outside =        78504/100000
        1000000: duration = 0.063138 seconds, pi = 3.1413, # inside/outside =       785318/1000000
       10000000: duration = 0.65805 seconds, pi = 3.1417, # inside/outside =      7854147/10000000
      100000000: duration = 9.6115 seconds, pi = 3.1419, # inside/outside =     78547002/100000000


Note that for larger `N`, `ray_estimate_pi_blocks` time scale noticeably slower than the original implementation, e.g., for the highest `N`, `100,000,000`, the durations are approximately `1.2` seconds vs. `9.6` seconds.

## 01 Ray Crash Course - Tasks - Exercise 2

What `N` value is needed to get a reliable estimate to five decimal places, `3.1415` (for some definition of "reliable")? If you have a powerful machine or a cluster, you could try a higher accuracy. You'll need to use the solution to Exercise 1 or you can make a guess based on the results we've already seen in this notebook.

To use the solution from Exercise 1, we'll need a modified `ray_try_it` to add the `m` blocks parameter:

In [100]:
def ray_try_it_blocks(n, m, trials):
    print('trials = {:5d}, N = {:s}: '.format(trials, str_large_n(n, padding=15)), end='')   # str_large_n imported above.
    start = time.time()
    ids = [ray_estimate_pi_blocks.remote(n, m) for _ in range(trials)]
    pis_counts = ray.get(ids)
    pis = list(map(lambda t: t[0], pis_counts))
    approx_pi = statistics.mean(pis)
    stdev = 0.0 if trials == 1 else statistics.stdev(pis)
    duration = time.time() - start
    error = (100.0*abs(approx_pi-np.pi)/np.pi)
    print(fmt.format(duration, approx_pi, stdev, error))   # str_large_n imported above.
    return trials, n, duration, approx_pi, stdev, error

Let's compute the error we would have to achieve for this accuracy. 

In [101]:
target_error = 100*abs(3.1415 - np.pi)/np.pi
target_error

0.002949255362150871

Okay, let's keep trying bigger `N` until we get to this number, but now we need to pick a definition of "reliable", because the results will depend on the number of `trials` we do. Also, some experiments will get "lucky" for relatively low `N` values.

> **WARNING:** This could take a while. You could choose a less accurate error goal if you have limited compute resources.

In [105]:
N = 100
error = 10.0
while error > target_error:
    N *= 10
    _, _, duration, approx_pi, _, error = ray_try_it_blocks(N, 1000000, trials)
    if N > 100000000:
        print("Stopping so we don't crash the machine...")
        break
print(f'{N} samples is sufficient to get the error below {target_error}%')

trials =     5, N =           1,000:    0.00670 seconds: pi ~ 3.151200, stddev = 0.0134, error = 0.3058%
trials =     5, N =          10,000:    0.00622 seconds: pi ~ 3.143440, stddev = 0.0188, error = 0.0588%
trials =     5, N =         100,000:    0.01528 seconds: pi ~ 3.142992, stddev = 0.0049, error = 0.0445%
trials =     5, N =       1,000,000:    0.09161 seconds: pi ~ 3.141640, stddev = 0.0016, error = 0.0015%
1000000 samples is sufficient to get the error below 0.002949255362150871%


You should run the previous cell several times. Some runs might succeed with `N = 100,000`, while more often it will be above 1M or 10M.

## 01 Ray Crash Course - Tasks - Exercise 3

For small computation problems, Ray adds enough overhead that its benefits are outweighed. You can see from the performance graphs in the lesson that smaller `N` or smaller trial values will likely cause the performance curves to cross. Try small values of `N` and small trial numbers. When do the lines cross? Try timing individual runs for small `N` around the crossing point. What can you infer from this "tipping point" about appropriate sizing of tasks, at least for your test environment?

First, here is more code from the notebook. Here is `try_it`, modified to handle the extra return values from the modified `estimate_pi`:

In [112]:
def try_it(n, trials):
    print('trials = {:3d}, N = {:s}: '.format(trials, str_large_n(n, padding=12)), end='')   # str_large_n imported above.
    start = time.time()
    pis_counts = [estimate_pi(n) for _ in range(trials)]
    pis = list(map(lambda t: t[0], pis_counts))
    approx_pi = statistics.mean(pis)
    stdev = statistics.stdev(pis)
    duration = time.time() - start
    error = (100.0*abs(approx_pi-np.pi)/np.pi)
    print(fmt.format(duration, approx_pi, stdev, error))   # str_large_n imported above.
    return trials, n, duration, approx_pi, stdev, error

In [135]:
small_ns = [1, 10, 100, 1000, 10000, 100000]

In [136]:
data_ns = [try_it(n, trials) for n in small_ns]

trials =   5, N =            1:    0.00038 seconds: pi ~ 3.200000, stddev = 1.7889, error = 1.8592%
trials =   5, N =           10:    0.00028 seconds: pi ~ 3.120000, stddev = 0.1789, error = 0.6873%
trials =   5, N =          100:    0.00032 seconds: pi ~ 3.080000, stddev = 0.0748, error = 1.9606%
trials =   5, N =        1,000:    0.00212 seconds: pi ~ 3.130400, stddev = 0.0661, error = 0.3563%
trials =   5, N =       10,000:    0.00213 seconds: pi ~ 3.139040, stddev = 0.0188, error = 0.0813%
trials =   5, N =      100,000:    0.01813 seconds: pi ~ 3.141728, stddev = 0.0021, error = 0.0043%


In [137]:
ray_data_ns = [ray_try_it(n, trials) for n in small_ns]

trials =     5, N =               1:    0.00300 seconds: pi ~ 2.400000, stddev = 2.1909, error = 23.6056%
trials =     5, N =              10:    0.00241 seconds: pi ~ 3.040000, stddev = 0.2191, error = 3.2338%
trials =     5, N =             100:    0.00214 seconds: pi ~ 3.144000, stddev = 0.1345, error = 0.0766%
trials =     5, N =           1,000:    0.00243 seconds: pi ~ 3.102400, stddev = 0.0378, error = 1.2475%
trials =     5, N =          10,000:    0.00324 seconds: pi ~ 3.144480, stddev = 0.0118, error = 0.0919%
trials =     5, N =         100,000:    0.01032 seconds: pi ~ 3.140808, stddev = 0.0046, error = 0.0250%


In [138]:
np_data_ns         = np.array(data_ns)
np_ray_data_ns     = np.array(ray_data_ns)

In [139]:
from bokeh_util import two_lines_plot, means_stddevs_plot  # Some plotting utilities in `./bokeh_util.py`.
from bokeh.plotting import show, figure
from bokeh.layouts import gridplot

In [140]:
two_lines = two_lines_plot(
    "N vs. Execution Times (Smaller Is Better)", 'N', 'Time', 'No Ray', 'Ray', 
    np_data_ns[:,1], np_data_ns[:,2], np_ray_data_ns[:,1], np_ray_data_ns[:,2],
    x_axis_type='log', y_axis_type='log')
show(two_lines, plot_width=800, plot_height=400)

(If you can't see it, click [here](../../images/Pi-small-Ns-vs-times.png).)

Let's calculate the `N` where they cross:

In [142]:
for i in range(len(small_ns)):
    if data_ns[i] >= ray_data_ns[i]:
        print(f'Crossing point: N = {small_ns[i]}')

Crossing point: N = 100000


## 02 Ray Crash Course - Actors - Exercise 1

You are asked these questions about the `Counter` vs. `RayCounter` performance:

> Ignoring pause = 0, can you explain why the Ray times are almost, but slightly larger than the non-ray times consistently? Study the implementations for `ray_counter_trial` and `RayCounter`. What code is synchronous and blocking vs. concurrent? In fact, is there _any_ code that is actually concurrent when you have just one instance of `Counter` or `RayCounter`?

Here is `ray_counter_trial` again, with comments about concurrency vs. synchronous blocking calls:

In [1]:
def ray_counter_trial(count_to, num_counters = 1, pause = 0.01):
    print('ray:     count_to = {:5d}, num counters = {:4d}, pause = {:5.3f}: '.format(count_to, num_counters, pause), end='')
    start = time.time()
    final_count_futures = []
    # Actor instantiation blocks, but returns almost immediately. The actor creation overhead is low. It is a little bit larger
    # than normal class instantiation, but insignificant for overall performance.
    counters = [RayCounter.remote(pause) for _ in range(num_counters)]
    for i in range(num_counters):
        for n in range(count_to):
            counters[i].next.remote()                                # Nonblocking, so will be faster for long pause scenarios...
        final_count_futures.append(counters[i].get_count.remote())  
    ray.get(final_count_futures)                                     # but block until all invocations are finished!
    duration = time.time() - start
    print('time = {:9.5f} seconds'.format(duration))
    return count_to, num_counters, pause, duration

Both `next` methods in `Counter` and `RayCounter`, call `time.sleep(pause)` before completing, but for `RayCounter` it runs asynchronously, while it blocks for `Counter`. You do have to block to get the current count and if lots of async invocations of `next` are being processed, a call to `ray.get(actor.get_counter())` will block until all of them are finished.

Hence, the reason a single `RayCounter` instance never outperforms a `Counter` instance is because _all_ the code in `ray_counter_trial` becomes effectively _synchronous_ because of the single line `ray.get(final_count_futures)`. Since the Ray implementation has extra overhead for Ray, it will always take a little longer.

The real benefit is running many counters concurrently. `ray_counter_trial` does this seamlessly, while `counter_trial` remains fully synchronous.

At the end of the exercise is this statement and question:

> Once past zero pauses, the Ray overhead is constant. It doesn't grow with the pause time. Can you explain why it doesn't grow?

The Ray overhead doesn't change because the number of Ray-related invocations don't change as the pause time grows. We still use one counter instance and ten invocations of it. Hence the overhead is a constant, even though the method invocations will take longer to complete, depending on the `pause` value. 